In [1]:
from typing import List, Set, Tuple


class Instruction:
    
    def __init__(self, axis: str, value: int):
        self.axis = axis
        self.value = value
    
    @classmethod
    def from_raw(cls, raw_instr: str):
        p1, p2 = raw_instr.split("=")
        
        axis = p1[-1]
        assert axis in ("x", "y")
        
        value = int(p2)
        
        return cls(axis, value)
    
    def __repr__(self):
        return f"Instruction({self.axis}={self.value})"


class TransparentPaper:
    
    def __init__(self, dots: Set[Tuple[int, int]]):
        self.dots = dots
        
        self.max_x = 0
        self.max_y = 0
        
        for x, y in dots:
            self.max_x = max(x, self.max_x)
            self.max_y = max(y, self.max_y)

    
    @classmethod
    def from_raw_coords(cls, raw_coords: str):
        coords = set()
        
        for raw_coord in raw_coords.split():
            x, y = [int(c) for c in raw_coord.split(",")]
            coords.add((x, y))
        
        return cls(coords)
    
    def print_grid(self):
        if self.max_x > 200 or self.max_y > 200:
            print("Too large to print")
            return
        
        grid = []
        for y in range(self.max_y+1):
            row = []
            for x in range(self.max_x+1):
                if (x, y) in self.dots:
                    row.append("#")
                else:
                    row.append(".")
            grid.append(row)
        
        for row in grid:
            print("".join(row))
            
    def fold(self, instruction: Instruction) -> None:
        if instruction.axis == "y":
            self.fold_up(instruction.value)
        elif instruction.axis == "x":
            self.fold_left(instruction.value)
        else:
            ValueError("Unexpected axis!", instruction.axis)
            
    def fold_up(self, axis_value: int):
        new_dots = set()
        for x, y in self.dots:
            if y < axis_value:
                # This dot doesn't need to mirrored
                new_dots.add((x, y))
            elif y > axis_value:
                # This dot needs to be mirrored
                new_dots.add((x, axis_value-(y-axis_value)))

        self.dots = new_dots
        self.max_y = axis_value - 1
        
    
    def fold_left(self, axis_value: int):
        new_dots = set()
        for x, y in self.dots:
            if x < axis_value:
                # This dot doesn't need to mirrored
                new_dots.add((x, y))
            elif x > axis_value:
                # This dot needs to be mirrored
                new_dots.add((axis_value-(x-axis_value), y))

        self.dots = new_dots
        self.max_x = axis_value - 1

In [2]:
input_filename = "input.txt"

with open(input_filename) as input_file:
    raw_coords, raw_instructions = input_file.read().split("\n\n")
    
paper = TransparentPaper.from_raw_coords(raw_coords)
instructions = [Instruction.from_raw(raw_instr) 
                for raw_instr in raw_instructions.splitlines()]

In [3]:
for i, instruction in enumerate(instructions):
    paper.fold(instruction)
    if i == 0:
        print(f"Answer to Part 1: {len(paper.dots)} dots are visible after first fold")

Answer to Part 1: 618 dots are visible after first fold


In [4]:
print("Answer to Part 2:\n")
paper.print_grid()

Answer to Part 2:

.##..#....###..####.#..#.####.#..#.#..#.
#..#.#....#..#.#....#.#..#....#.#..#..#.
#..#.#....#..#.###..##...###..##...#..#.
####.#....###..#....#.#..#....#.#..#..#.
#..#.#....#.#..#....#.#..#....#.#..#..#.
#..#.####.#..#.####.#..#.#....#..#..##..
